In [69]:
import pandas as pd
import csv
import numpy as np
pd.set_option('display.max_rows', 1000)
from sklearn.metrics import classification_report

gold = pd.read_csv("./../data/conll/test_only_space-spans.conll",
                   sep="\t", quoting = csv.QUOTE_NONE, encoding="utf-8", dtype='str').dropna()
gold['End'] = gold['End'].apply(lambda x: int(x[:-2]))

pred = pd.read_csv("/home/amansinha/Downloads/test__transf_multilingual_crf_context_rnn_true.tsv",
                   sep=" ", names=['token', 'gold','pred'], quoting = csv.QUOTE_NONE, encoding="utf-8", dtype='str')

In [71]:
df = pd.concat([gold.reset_index(), pred], axis=1)
df

,index,Doc_Id,Sent_Id,Token_Id,Word,Tag,Begin,End,token,gold,pred
0,0,1091374539601309697,1,1,¿Qué,O,0,4,¿Qué,O,O
1,1,1091374539601309697,1,2,probabilidad,O,5,17,probabilidad,O,O
2,2,1091374539601309697,1,3,hay,O,18,21,hay,O,O
3,3,1091374539601309697,1,4,de,O,22,24,de,O,O
4,4,1091374539601309697,1,5,no,O,25,27,no,O,O
...,...,...,...,...,...,...,...,...,...,...,...
75695,78195,1500170784425136130,2500,23,no,O,112,114,no,O,O
75696,78196,1500170784425136130,2500,24,me,O,115,117,me,O,O
75697,78197,1500170784425136130,2500,25,logro,O,118,123,logro,O,O
75698,78198,1500170784425136130,2500,26,calmar,O,124,130,calmar,O,O


In [72]:
print(len(gold), len(pred))

75700 75700


In [73]:
print(classification_report(df.gold, df.pred, labels=["B-MISC","I-MISC"]))

              precision    recall  f1-score   support

      B-MISC       0.94      0.96      0.95      4259
      I-MISC       0.90      0.85      0.87      1570

   micro avg       0.93      0.93      0.93      5829
   macro avg       0.92      0.90      0.91      5829
weighted avg       0.93      0.93      0.93      5829



In [74]:
def peek_stack(stack):
    if stack:
        return stack[-1]    # this will get the last element of stack
    else:
        return None
    
def merge_spans2(spans, words, debug=False):
    
    if debug : print(spans)
    spans = np.asarray(spans)
    print(spans.shape)
    if spans.shape[0] == 1:
        return spans, words
    stack = []
    merged_words = []
    for s, w in zip(spans.reshape(1,-1)[0], words):
        if debug : print(s)
            
        if len(stack) == 0:
            stack.append(s)
            merged_words.append(w)
        else:
            if s - peek_stack(stack) == 1:
                stack.pop()
            else:
                stack.append(s)
        if debug : print("current stack: ", stack)
    stack = np.asarray(stack)
    return stack.reshape(-1, 2), merged_words

In [79]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import csv
import numpy as np
import argparse
from tqdm import tqdm

from sklearn.metrics import classification_report

    
#merge_spans2(debug=True)    

# def create_submission_file(args):

path = "./../data/socialdisner_v3/train-valid-txt-files/validation/"

validation = gold

predictions = pred

# validation['pred'] = predictions.pred

print(classification_report(predictions.gold, predictions.pred, labels=['B-MISC', 'I-MISC']))

print('converting to submission format')
with open("./../submissions/testpred_mana_with_mergingfix.tsv", "w", encoding="utf-8") as fp:
    print('tweets_id','begin','end','type','extraction',sep='\t', file=fp)
    for i in tqdm(range(1, len(validation.Sent_Id.drop_duplicates()) + 1)):
        info = df[df.Sent_Id.astype(int) == i]
        #print(info)
        l = info[info.pred != 'O']
        print(l)
        if len(l) == 0: continue
        docid = l.Doc_Id.values[0]
        print(docid)
        f = open(f'{path}{docid}.txt', 'r', encoding="utf-8")
        # replace tab with space then strip extra spaces
        text = f.read().replace('\n',' ').strip()
        annoying_chars = ["️","⁩","⁦","2⃣"," ", " ", " ", "⁩", "⁦", "‍", " "]
        for char in annoying_chars:
            text = text.replace(char, '_').strip()
        #print(text)
        #print('*'*20)
        #print(l.Doc_Id.values[0], l.begin.values[0], l.end.values[0], l.pred.values[0])
        spans = [[s,e] for s,e in zip(l.Begin.astype(int), l.End.astype(int))]
        words = [w for w in l.Word]
        tags = [t for t in l.pred]
        print(spans, words)
        #spans,mwords = merge_spans3(spans, words)
        spans,mwords = merge_spans4(spans, words, tags)
        #print(spans)
        for span,mw in zip(spans,mwords):
#             aman = text[span[0]: span[1]]
#             for char in annoying_chars:
#                 aman = aman.replace(char, '_').strip()
            print(docid, span[0], span[1], 'ENFERMEDAD', mw, sep='\t', file=fp)

#         "print("#"*20)
        #break


# create_submission_file(args)

  0%|          | 12/2500 [00:00<00:21, 118.40it/s]

              precision    recall  f1-score   support

      B-MISC       0.94      0.96      0.95      4259
      I-MISC       0.90      0.85      0.87      1570

   micro avg       0.93      0.93      0.93      5829
   macro avg       0.92      0.90      0.91      5829
weighted avg       0.93      0.93      0.93      5829

converting to submission format
   index               Doc_Id Sent_Id Token_Id   Word     Tag Begin  End  \
7      7  1091374539601309697       1        8  gripe  B-MISC    38   43   

   token    gold    pred  
7  gripe  B-MISC  B-MISC  
1091374539601309697
[[38, 43]] ['gripe']
    index               Doc_Id Sent_Id Token_Id          Word     Tag Begin  \
28     29  1091997525605826560       2       13  discapacidad  B-MISC    77   

    End         token    gold    pred  
28   89  discapacidad  B-MISC  B-MISC  
1091997525605826560
[[77, 89]] ['discapacidad']
    index               Doc_Id Sent_Id Token_Id      Word     Tag Begin  End  \
77     79  109236354501728

  1%|▏         | 36/2500 [00:00<00:21, 116.16it/s]

     index               Doc_Id Sent_Id Token_Id            Word     Tag  \
681    704  1099253802505588736      24        9  #bronquiolitis  B-MISC   
682    705  1099253802505588736      24       10           VRS+.  I-MISC   

    Begin  End           token    gold    pred  
681    44   58  #bronquiolitis  B-MISC  B-MISC  
682    59   64           VRS+.  I-MISC  I-MISC  
1099253802505588736
[[44, 58], [59, 64]] ['#bronquiolitis', 'VRS+.']
     index               Doc_Id Sent_Id Token_Id               Word     Tag  \
710    734  1100126807674757122      25        4           #autismo  B-MISC   
712    736  1100126807674757122      25        6  hipersensibilidad  B-MISC   
713    737  1100126807674757122      25        7         sensorial.  I-MISC   

    Begin  End              token    gold    pred  
710    15   23           #autismo  B-MISC  B-MISC  
712    26   43  hipersensibilidad  B-MISC  B-MISC  
713    44   54         sensorial.  I-MISC  I-MISC  
1100126807674757122
[[15, 23],

  2%|▏         | 61/2500 [00:00<00:21, 114.34it/s]

...

       index               Doc_Id Sent_Id Token_Id              Word     Tag  \
74145  76594  1495698137649983493    2449       12         alzheimer  B-MISC   
74150  76599  1495698137649983493    2449       17        #Alzheimer  B-MISC   
74158  76607  1495698137649983493    2449       25  @FEVAFAAlzheimer  B-MISC   

      Begin  End             token    gold    pred  
74145    62   71         alzheimer  B-MISC  B-MISC  
74150   125  135        #Alzheimer  B-MISC  B-MISC  
74158   223  239  @FEVAFAAlzheimer  B-MISC  B-MISC  
1495698137649983493
[[62, 71], [125, 135], [223, 239]] ['alzheimer', '#Alzheimer', '@FEVAFAAlzheimer']
       index               Doc_Id Sent_Id Token_Id      Word     Tag Begin  \
74209  76659  1495730711785095170    2450       51  ansiedad  B-MISC   269   

       End     token    gold    pred  
74209  277  ansiedad  B-MISC  B-MISC  
1495730711785095170
[[269, 277]] ['ansiedad']
       index               Doc_Id Sent_Id Token_Id   Word     Tag Begin  End  \
742

 99%|█████████▉| 2481/2500 [00:22<00:00, 94.03it/s]

       index               Doc_Id Sent_Id Token_Id      Word     Tag Begin  \
74796  77266  1496576024884260868    2470        7  #COVID19  B-MISC    31   

       End     token    gold    pred  
74796   39  #COVID19  B-MISC  B-MISC  
1496576024884260868
[[31, 39]] ['#COVID19']
       index               Doc_Id Sent_Id Token_Id      Word     Tag Begin  \
74809  77280  1496591921376018436    2471       11  #COVID19  B-MISC    52   
74830  77301  1496591921376018436    2471       32      #SAS       O   186   
74831  77302  1496591921376018436    2471       33      #HCS       O   191   

       End     token    gold    pred  
74809   60  #COVID19  B-MISC  B-MISC  
74830  190      #SAS       O  B-MISC  
74831  195      #HCS       O  B-MISC  
1496591921376018436
[[52, 60], [186, 190], [191, 195]] ['#COVID19', '#SAS', '#HCS']
       index               Doc_Id Sent_Id Token_Id      Word     Tag Begin  \
74847  77319  1496599479603982336    2472       14  ansiedad  B-MISC    62   

       End 

100%|██████████| 2500/2500 [00:22<00:00, 109.31it/s]

       index               Doc_Id Sent_Id Token_Id      Word     Tag Begin  \
75318  77807  1498269003877335040    2489       15  ansiedad  B-MISC    78   

       End     token    gold    pred  
75318   86  ansiedad  B-MISC  B-MISC  
1498269003877335040
[[78, 86]] ['ansiedad']
       index               Doc_Id Sent_Id Token_Id     Word     Tag Begin  \
75355  77845  1498364279807950848    2490       37  cáncer,  B-MISC   176   

       End    token    gold    pred  
75355  183  cáncer,  B-MISC  B-MISC  
1498364279807950848
[[176, 183]] ['cáncer,']
       index               Doc_Id Sent_Id Token_Id     Word     Tag Begin  \
75376  77867  1498605286638166019    2491        3    cacho       O     6   
75377  77868  1498605286638166019    2491        4       de       O    12   
75378  77869  1498605286638166019    2491        5    riñón  B-MISC    15   
75381  77872  1498605286638166019    2491        8  tumores  I-MISC    32   

       End    token    gold    pred  
75376   11    cacho  

In [7]:
import numpy as np
def merge_spans3(spans=[[39, 46], [47, 52], [55, 62]], words = ['bcbc', 'aa', 'autismo'], debug=False):
    
    if debug : print(spans)
    spans = np.asarray(spans)
    stack = []
    tag_stack = []
    s2w = {}
    
    for s in spans.reshape(1,-1)[0]:
        
        if debug : print('==>', s)
        if len(stack) == 0:
            stack.append(s)
        else:
            if s - peek_stack(stack) == 1:
                stack.pop()
            else:
                stack.append(s)
        if debug : print("current stack: ", stack)
    stack = np.asarray(stack).reshape(-1,2)
    
    for s,word in zip(spans,words):
        for ss in s:
            s2w[ss] = word
    word_list = []
    for s in stack:
        sw = []
        #print(s[0], s[1])
        idx = [n for n in s2w.keys() if (n>=s[0] and n<=s[1])]
        widx = [s2w[i] for i in idx]
        fin_word = []
        for w in widx :
            if w not in fin_word:
                fin_word.append(w)
        #print(widx)
        print(s, ' '.join(fin_word))
        word_list.append(' '.join(fin_word))
    
    print(len(stack), len(word_list))
        
    return stack, word_list
    
merge_spans3(debug=True)

[[39, 46], [47, 52], [55, 62]]
==> 39
current stack:  [39]
==> 46
current stack:  [39, 46]
==> 47
current stack:  [39]
==> 52
current stack:  [39, 52]
==> 55
current stack:  [39, 52, 55]
==> 62
current stack:  [39, 52, 55, 62]
[39 52] bcbc aa
[55 62] autismo
2 2


(array([[39, 52],
        [55, 62]]),
 ['bcbc aa', 'autismo'])

In [40]:
def merge_spans4(spans=[[39, 46], [47, 54], [55, 62]], words = ['bcbc', 'aa', 'autismo'], tags = ['B-MISC', 'I-MISC', 'I-MISC'], debug=False):
    
    if debug : print(spans)
    spans = np.asarray(spans)
    stack = []
    tag_stack = []
    s2w = {}
    
    span2tag = {}
    for i,(s,t) in enumerate(zip(spans, tags)):
        for ss in s:
            span2tag[ss] = t+f'_{i}'
    if debug : print(span2tag)
    
    for s in spans.reshape(1,-1)[0]:
        
        if debug : print('==>', s)
        if len(stack) == 0:
            stack.append(s)
        else:
            # if stack is not empty
            if debug: print(span2tag[peek_stack(stack)], span2tag[s])
            # and if stack top has not same tag id 
            
            # posibilities : bi ib bb01 ii01 bi ib
            
            if span2tag[peek_stack(stack)] != span2tag[s]:
            # two case : (bmisc0 and bmisc1) or (bmsic0 imsc1) or (imsic0 imsc1)
                if debug: print("stack top tag id different: ",span2tag[peek_stack(stack)][:-2], span2tag[s][:-2])
                #if span2tag[peek_stack(stack)][:-2] ==  'B-MISC' and span2tag[peek_stack(stack)][:-2] == 'I-MISC':
                #if span2tag[peek_stack(stack)][:-2] ==  'I-MISC' and span2tag[peek_stack(stack)][:-2] == 'I-MISC':
                if debug: print("now we have incoming:" , span2tag[s][:-2])
                if  span2tag[s][:-2] == 'I-MISC':
                    if debug: print('inside:' , peek_stack(stack), 'to add:', s)
                    if s - peek_stack(stack) == 1:
                        # character difference is fine
                        if debug: print(".....merging by removing the top of the stack !!\n")
                        stack.pop()
                    else:
                        stack.append(s)
                
                elif span2tag[peek_stack(stack)][:-2] ==  'B-MISC' and span2tag[s][:-2] == 'B-MISC':
                    stack.append(s)
                elif span2tag[peek_stack(stack)][:-2] ==  'I-MISC' and span2tag[s][:-2] == 'B-MISC':
                    stack.append(s)
                    
            else:
                # bb00 ii00
                # if stack top has same tag id
                # i.e. part of the same span
                stack.append(s)
        
        
        
        if debug : print("current stack: ", stack, "*"*20, '\n')
    stack = np.asarray(stack).reshape(-1,2)
    
    for s,word in zip(spans,words):
        for ss in s:
            s2w[ss] = word
    word_list = []
    for s in stack:
        sw = []
        #print(s[0], s[1])
        idx = [n for n in s2w.keys() if (n>=s[0] and n<=s[1])]
        widx = [s2w[i] for i in idx]
        fin_word = []
        for w in widx :
            if w not in fin_word:
                fin_word.append(w)
        #print(widx)
        if debug: print(s, ' '.join(fin_word))
        word_list.append(' '.join(fin_word))
    
    if debug: print(len(stack), len(word_list))
        
    return stack, word_list
    
merge_spans4(debug=True)

[[39, 46], [47, 54], [55, 62]]
{39: 'B-MISC_0', 46: 'B-MISC_0', 47: 'I-MISC_1', 54: 'I-MISC_1', 55: 'I-MISC_2', 62: 'I-MISC_2'}
==> 39
current stack:  [39] ******************** 

==> 46
B-MISC_0 B-MISC_0
current stack:  [39, 46] ******************** 

==> 47
B-MISC_0 I-MISC_1
stack top tag id different:  B-MISC I-MISC
now we have incoming: I-MISC
inside: 46 to add: 47
.....merging by removing the top of the stack !!

current stack:  [39] ******************** 

==> 54
B-MISC_0 I-MISC_1
stack top tag id different:  B-MISC I-MISC
now we have incoming: I-MISC
inside: 39 to add: 54
current stack:  [39, 54] ******************** 

==> 55
I-MISC_1 I-MISC_2
stack top tag id different:  I-MISC I-MISC
now we have incoming: I-MISC
inside: 54 to add: 55
.....merging by removing the top of the stack !!

current stack:  [39] ******************** 

==> 62
B-MISC_0 I-MISC_2
stack top tag id different:  B-MISC I-MISC
now we have incoming: I-MISC
inside: 39 to add: 62
current stack:  [39, 62] ***********

(array([[39, 62]]), ['bcbc aa autismo'])